Before beginning to run todays notebooks, please run the following commands in the terminal:

```ln -s /datasets/bg237-wi22-a00-public/module-10-popgen-data/data1/ ~/module-10-popgen/workdir1/data```

```ln -s /datasets/bg237-wi22-a00-public/module-10-popgen-data/data2/ ~/module-10-popgen/workdir2/data```

In [ ]:
require( data.table)

In [ ]:
getwd()

In [ ]:
list.files()

## Prep data for analysis

In [ ]:
ped.clean <- fread( './data/KGP_GSA_postQC.ped' )
map.clean <- fread( './data/KGP_GSA_postQC.map' )

In [ ]:
dim( ped.clean )
dim( map.clean )

In [ ]:
names( map.clean ) <- c( "CHR", "SNP", "cM", "BP" )

In [ ]:
names( ped.clean )[ 1:6 ] <- c( "FID", "IID", "MID", "PID", "Sex", "Pheno" )  
names( ped.clean )[ 7:dim(ped.clean)[2] ] <- map.clean$SNP

In [ ]:
ped.clean[ 1:10, 1:11 ]
map.clean[ 1:5, ]

#### Look at a few SNPs

In [ ]:
table( ped.clean$rs707582 )
table( ped.clean$rs6462740 )
table( ped.clean$rs11740127 )
table( ped.clean$rs17114046 )

In [ ]:
# We need numbers, not letters, to do stats.  We like the additive model.
# We need to count one of the alleles for each SNP, to get a number for analysis.
# We can count either allele, or we could code as dominant or overe dominant.
# For now, let's use a simple additive model.  Count whichever allele you fancy.

ped.clean.add <- ped.clean

In [ ]:
## Code Data according to an additive model
# fill in the SNP name: rs707582
table( ped.clean$ )

In [ ]:
# Code rs707582 according to an additive model
# fill in the SNP name: rs707582
# fill in the genotype codes to specify the counts
ped.clean.add$[ ped.clean.add$ == "" ] <- 0
ped.clean.add$[ ped.clean.add$ == "" ] <- 1
ped.clean.add$[ ped.clean.add$ == "" ] <- 2

In [ ]:
#fill in the SNP name: rs707582 to compare new coding to old coding
table( ped.clean.add$, ped.clean$ )

In [ ]:
# Update all SNPs

map.clean$A1 <- NA
map.clean$A2 <- NA
ped.clean.add <- ped.clean

In [ ]:
for ( snp in map.clean$SNP ) {
	
	i <- which( names( ped.clean.add ) == snp )
	j <- which( map.clean$SNP == snp )
	 
	temp <- paste( ped.clean[[ i ]], collapse=' ' )
	alleles <- unique( strsplit( temp, split=' ' )[[1]] )

	map.clean$A1[ j ] <- alleles[ 1 ]
	map.clean$A2[ j ] <- alleles[ 2 ]

	ped.clean.add[[i]][ ped.clean[[i]] == paste( map.clean$A1[j], map.clean$A1[j] ) ] <- 2
	ped.clean.add[[i]][ ped.clean[[i]] == paste( map.clean$A1[j], map.clean$A2[j] ) ] <- 1
	ped.clean.add[[i]][ ped.clean[[i]] == paste( map.clean$A2[j], map.clean$A1[j] ) ] <- 1
	ped.clean.add[[i]][ ped.clean[[i]] == paste( map.clean$A2[j], map.clean$A2[j] ) ] <- 0
	ped.clean.add[[i]] <- as.numeric( ped.clean.add[[i]] )		
								
}

In [ ]:
ped.clean.add[ 1:10, 1:11 ]
map.clean[ 1:5, ]

#### Look at a few SNPs

In [ ]:
map.clean[ map.clean$SNP == 'rs707582' ]
table( ped.clean$rs707582, ped.clean.add$rs707582 )

map.clean[ map.clean$SNP == 'rs6462740' ]
table( ped.clean$rs6462740, ped.clean.add$rs6462740 )

map.clean[ map.clean$SNP == 'rs11740127' ]
table( ped.clean$rs11740127, ped.clean.add$rs11740127 )

map.clean[ map.clean$SNP == 'rs17114046' ]
table( ped.clean$rs17114046, ped.clean.add$rs17114046 )

## Load phenotype data

In [ ]:
# Case control trait
pheno.cc <- fread( './data/CaseControl_pheno_all.txt' )

In [ ]:
# Quantitative trait
pheno.qt <- fread( './data/QuantitativeTrait_pheno_all.txt' )

#### Visualize distributions

In [ ]:
hist( pheno.qt$qTrait, breaks='fd' )

hist( pheno.cc$ccTrait, breaks='fd' )

## Compare single SNP with phenotypes

In [ ]:
# Test SNPs: rs707582, rs9913145, rs7237102, rs7741565

#### Quantitative trait

In [ ]:
# Add phenotype to ped matrix
ped.clean.add.qt <- ped.clean.add
ped.clean.add.qt$Pheno <- pheno.qt$qTrait[ match( pheno.qt$IID, ped.clean.add$IID ) ]

In [ ]:
# Make a descriptive plot
plot( ped.clean.add.qt$rs707582, ped.clean.add.qt$Pheno )
# Mean of 0's
points( 0, mean( ped.clean.add.qt$Pheno[ ped.clean.add.qt$rs707582 == 0 ] ), col=2, pch=15 )
# Mean of 1's
points( 1, mean( ped.clean.add.qt$Pheno[ ped.clean.add.qt$rs707582 == 1 ] ), col=2, pch=15 )
# Mean of 2's
points( 2, mean( ped.clean.add.qt$Pheno[ ped.clean.add.qt$rs707582 == 2 ] ), col=2, pch=15 )

#Line from mean of 0's to mean of 2's -> Why? What are we looking for?
segments( 0, mean( ped.clean.add.qt$Pheno[ ped.clean.add.qt$rs707582 == 0 ] ),
			2, mean( ped.clean.add.qt$Pheno[ ped.clean.add.qt$rs707582 == 2 ] ), 
			col=2 )

#### Linear regression

In [ ]:
# rs707582, rs9913145, rs7237102, rs7741565
snp.lm <- lm( Pheno ~ rs707582, data=ped.clean.add.qt )

In [ ]:
summary( snp.lm )

In [ ]:
# Make a descriptive plot
plot( ped.clean.add.qt$rs707582, ped.clean.add.qt$Pheno )
# Mean of 0's
points( 0, mean( ped.clean.add.qt$Pheno[ ped.clean.add.qt$rs707582 == 0 ] ), col=2, pch=15 )
# Mean of 1's
points( 1, mean( ped.clean.add.qt$Pheno[ ped.clean.add.qt$rs707582 == 1 ] ), col=2, pch=15 )
# Mean of 2's
points( 2, mean( ped.clean.add.qt$Pheno[ ped.clean.add.qt$rs707582 == 2 ] ), col=2, pch=15 )
#Line from mean of 0's to mean of 2's -> Why? What are we looking for?
segments( 0, mean( ped.clean.add.qt$Pheno[ ped.clean.add.qt$rs707582 == 0 ] ),
				2, mean( ped.clean.add.qt$Pheno[ ped.clean.add.qt$rs707582 == 2 ] ), 
				col=2 )
# Add regression line
abline( snp.lm, col='blue' )

In [ ]:
# Try a few random SNPs 
# How good is our additive model fit our data? 
# What about the "pure additive model"?

# rs707582
# rs9913145
# rs7237102
# rs7741565

#### Binary trait ( cases=2, controls=1 )

In [ ]:
ped.clean.add.cc <- ped.clean.add
ped.clean.add.cc$Pheno <- pheno.cc$ccTrait[ match( pheno.cc$IID, ped.clean.add$IID ) ]

map.clean[ map.clean$SNP == 'rs707582' ]

table( ped.clean.add.cc$Pheno, ped.clean.add.cc$rs707582 )

In [ ]:
# 2x3 table - but we are interested in an additive _allelic_ model ...
#
#			A1	A2
#	case	a	c
#	control	b	d
#
#	odds | A1: a/b
#	odds | A2: c/d 

a <- 2* + 1*
b <- 2* + 1*
c <- 2* + 1*
d <- 2* + 1*

add.allele.table <- matrix( c( a,b,c,d ), 2, 2 )
add.allele.table


In [ ]:
## Is there a relationship?
## what is the odds ratio?

or <- ( a*d ) / ( b*c )
or

In [ ]:
#	rs707582
#		A1	A2
#	ca	637	393
#	co	793	505
#
# a <- 2*191 + 1*255
# b <- 2*234 + 1*325
# c <- 2*69 + 1*255
# d <- 2*90 + 1*325
#
# or <- ( a*d ) / ( b*c )
# or
#
#	or = 1.03

# rs9913145
# rs7237102
# rs7741565

In [ ]:
# Chi square test for independence:
#
#			A1	A2
#	case	a	c
#	control	b	d
#
# chi.sq = sum( ( o_i - e_i)^2 / e_i )
# 
# P( X and Y | X,Y are independent ) = P(X)P(Y)
# E( X ) = N*P(X)
#
# E(a under null)    = E( A1 and case | A1 and case are independent )  
#                    = n * P(A1) * P(Case)
#                    = (a+b+c+d) * (a+b)/(a+b+c+d) * (a+c)/(a+b+c+d)

n <- a+b+c+d
e_a <- n * (a+b)/n * (a+c)/n
e_b <- n * (a+b)/n * (b+d)/n
e_c <- n * (c+d)/n * (a+c)/n
e_d <- n * (c+d)/n * (b+d)/n

chi2 <- ( a-e_a )^2/e_a + ( b-e_b )^2/e_b + ( c-e_c )^2/e_c + ( d-e_d )^2/e_d
pchisq( chi2, 1, lower.tail=F )

#### Logistic regression

In [ ]:
# We can't use a lm() for a binary outcomes, but we can use a logistic regression
# in R this is fitted using glm() with the family='binomial' option
ped.clean.add.cc$Pheno <- ped.clean.add.cc$Pheno-1

snp.logm <- glm( Pheno ~ rs707582, data=ped.clean.add.cc, family='binomial' )

summary( snp.logm )

In [ ]:
# How does our OR compare to the ln(OR) from logistic regression?
# Try a few others
# There are some violated assumptions so the 'direct or' and 'logsitc or'
#    don't quite add up.  More on that later ...

# rs707582
# rs9913145
# rs7237102
# rs7741565

## How about a Genome-Wide Association Study (GWAS)

In [ ]:
# We want one p-value per SNP, per phenotype
# Let's get the idea from our mini data set.
# Let's store the p's in our map file for easy access.
map.clean$p.qt <- NA
map.clean$p.cc <- NA
map.clean$p.rand <- NA

In [ ]:
set.seed( 123456 )   #So we all get the same "random vector"
ped.clean.add.rand <- ped.clean.add.qt
ped.clean.add.rand$Pheno <- rnorm( length( ped.clean.add.rand$Pheno ) )

In [ ]:
for ( snp in map.clean$SNP ) {

	temp.qt <- as.formula( paste0( 'Pheno ~ ', snp ) )
	temp.lm <- lm( temp.qt, data=ped.clean.add.qt )
 	map.clean$p.qt[ map.clean$SNP == snp ] <- summary( temp.lm )$coefficient[ 2,4 ]

	temp.cc <- as.formula( paste0( 'Pheno ~ ', snp ) )
	temp.glm <- glm( temp.cc, data=ped.clean.add.cc, family='binomial' )
 	map.clean$p.cc[ map.clean$SNP == snp ] <- summary( temp.glm )$coefficient[ 2,4 ]

	temp.r <- as.formula( paste0( 'Pheno ~ ', snp ) )
	temp.glm <- lm( temp.r, data=ped.clean.add.rand )
 	map.clean$p.rand[ map.clean$SNP == snp ] <- summary( temp.glm )$coefficient[ 2,4 ]
 	
}

#### How did we do?

In [ ]:
sum( map.clean$p.rand < 0.05 )
sum( map.clean$p.rand < 5e-8 )

In [ ]:
sum( map.clean$p.qt < 0.05 )
sum( map.clean$p.qt < 5e-8 )

In [ ]:
sum( map.clean$p.cc < 0.05 )
sum( map.clean$p.cc < 5e-8 )

#### Random phenotype

In [ ]:
# Mini manhattan plot

In [ ]:
# Some plotting positions
map.clean$ManPos <- cumsum( map.clean$BP/1000 )
chr.mids <- cbind( 1:22, NA )
for ( i in 1:22 ) {
	chr.start <- min( map.clean$ManPos[ map.clean$CHR == i ] )	
	chr.stop <- max( map.clean$ManPos[ map.clean$CHR == i ] )
	chr.length <- chr.stop-chr.start
	chr.mids[ i,2 ] <- chr.start + chr.length/2
}

In [ ]:
# Plot random phenotype p-values in Manhattan plot
plot( map.clean$ManPos, -log10( map.clean$p.rand ),
		xaxt='n', main="random", 
		col=c( 'black','grey' )[ map.clean$CHR %% 2 + 1 ],
		xlab="Genome Position", ylab="-log10(p)",
		ylim=c( 0,10 ) )
abline( h=-log10( 0.05 ), col='green' )
abline( h=-log10( 5e-8 ), col='green' )
axis( 1, at=chr.mids[,2], labels=chr.mids[,1] )

In [ ]:
# Plot random phenotype p-values in QQ plot
observed.p <- -log10( map.clean$p.rand[ order( map.clean$p.rand ) ] )
expected.p <- -log10( ( 1:length( map.clean$p.rand ) ) / ( length( map.clean$p.rand ) + 1 ) )
plot( expected.p, observed.p, main="random" )
abline( 0,1 )

## qt

In [ ]:
# Plot simulated QT phenotype p-values in Manhattan plot
plot( map.clean$ManPos, -log10( map.clean$p.qt ),
		xaxt='n', main="linear", 
		col=c( 'black','grey' )[ map.clean$CHR %% 2 + 1 ],
		xlab="Genome Position", ylab="-log10(p)" )
abline( h=-log10( 0.05 ), col='green' )
abline( h=-log10( 5e-8 ), col='green' )
axis( 1, at=chr.mids[,2], labels=chr.mids[,1] )

In [ ]:
# Plot simulated QT phenotype p-values in QQ plot
observed.p <- -log10( map.clean$p.qt[ order( map.clean$p.qt ) ] )
expected.p <- -log10( ( 1:length( map.clean$p.qt ) ) / ( length( map.clean$p.qt ) + 1 ) )
plot( expected.p, observed.p, main="linear" )
abline( 0,1 )

In [ ]:
## What do we think of these plots?
## Does this look good or problematic?

## cc

In [ ]:
# Plot simulated CC phenotype p-values in Manhattan plot
plot( map.clean$ManPos, -log10( map.clean$p.cc ),
		xaxt='n', main="case-control", 
		col=c( 'black','grey' )[ map.clean$CHR %% 2 + 1 ],
		xlab="Genome Position", ylab="-log10(p)" )
abline( h=-log10( 0.05 ), col='green' )
abline( h=-log10( 5e-8 ), col='green' )
axis( 1, at=chr.mids[,2], labels=chr.mids[,1] )

In [ ]:
# Plot simulated QT phenotype p-values in QQ plot
observed.p <- -log10( map.clean$p.cc[ order( map.clean$p.cc ) ] )
expected.p <- -log10( ( 1:length( map.clean$p.cc ) ) / ( length( map.clean$p.cc ) + 1 ) )
plot( expected.p, observed.p, main="case-control" )
abline( 0,1 )

In [ ]:
## What do we think of these plots?
## Does this look good or problematic?

## Here is the full genome results:

In [ ]:
sumstats <- fread( './data/ALL_gwas.assoc.linear' )

In [ ]:
dim( sumstats )
head( sumstats )

In [ ]:
sum( sumstats$P < 0.05 )
sum( sumstats$P < 0.05 ) / dim( sumstats )[1]
sum( sumstats$P < 5e-8 )
sum( sumstats$P < 5e-8 ) / dim( sumstats )[1]

In [ ]:
# Qick Manhattan
sumstats$ManPos <- cumsum( sumstats$BP/1000 )
chr.mids <- cbind( 1:22, NA )
for ( i in 1:22 ) {
	chr.start <- min( sumstats$ManPos[ sumstats$CHR == i ] )
	chr.stop <- max( sumstats$ManPos[ sumstats$CHR == i ] )
	chr.length <- chr.stop-chr.start
	chr.mids[ i,2 ] <- chr.start + chr.length/2
}

In [ ]:
#####ERROR#####
plot( sumstats$ManPos[ sumstats$P < 0.1 ],
		-log10( sumstats$P[ sumstats$P < 0.1 ] ),
		xaxt='n', pch=16,
		col=c( 'black','grey' )[ sumstats$CHR %% 2 + 1 ],
		xlab="Genome Position", ylab="-log10(p)",
		ylim=c(0,max( -log10( sumstats$P ) )+1 ) )
abline( h=-log10( 0.05 ), col='green' )
abline( h=-log10( 5e-8 ), col='green' )
axis( 1, at=chr.mids[,2], labels=chr.mids[,1] )

In [ ]:
# Qick qq-plot
observed.p <- -log10( sumstats$P[ order( sumstats$P ) ] )
expected.p <- -log10( ( 1:length( sumstats$P ) ) / ( length( sumstats$P ) ) )
plot( expected.p[ expected.p < 4 ], observed.p[ expected.p < 4 ], 
		type='l', lwd=6,
		xlim=c( 0, max(expected.p)+1 ), ylim=c( 0, max(observed.p)+1 ) )
points( expected.p[ expected.p >= 4 ], observed.p[ expected.p >= 4 ],
		pch=16 )
abline( 0,1 )

In [ ]:
## What do we think of these plots?
## Does this look good or problematic?

## Population structure analysis

In [ ]:
## There is a hint in our ped file ...
head( ped.clean.add.cc )[ ,1:10 ]
table( ped.clean.add.cc$FID )

In [ ]:
# This data set involved real genotypes on people from the 1000 genomes project
# The goal was to sample human diversity and so contains people from multipl
# ancetsries.  Here, AFR=African, EUR=European - both of which contain subgroups
# See:  https://www.internationalgenome.org/data-portal/population

In [ ]:
# Does the phenotype vary with ancestry?

table( ped.clean.add.cc$FID, ped.clean.add.cc$Pheno )

In [ ]:
# Is there an association between ancestry and case-control? 
# Hint: calculate an odds ratio
a <- 
b <- 
c <- 
d <- 

(a*d) / (b*c)

In [ ]:
## Chi sq test

n <- a+b+c+d
e_a <- n * (a+b)/n * (a+c)/n
e_b <- n * (a+b)/n * (b+d)/n
e_c <- n * (c+d)/n * (a+c)/n
e_d <- n * (c+d)/n * (b+d)/n

chi2 <- ( a-e_a )^2/e_a + ( b-e_b )^2/e_b + ( c-e_c )^2/e_c + ( d-e_d )^2/e_d
pchisq( chi2, 1, lower.tail=F )

In [ ]:
#Are the means of the qt different?

par( mfcol=c(2,1) )
hist( ped.clean.add.qt$Pheno[ ped.clean.add.cc$FID == 'AFR' ],
		breaks=seq(-5,5,by=0.5) )
abline( v=mean( ped.clean.add.qt$Pheno[ ped.clean.add.cc$FID == 'AFR' ] ), col='red' )
hist( ped.clean.add.qt$Pheno[ ped.clean.add.cc$FID == 'EUR' ],
		breaks=seq(-5,5,by=0.5) )
abline( v=mean( ped.clean.add.qt$Pheno[ ped.clean.add.cc$FID == 'EUR' ] ), col='red' )

In [ ]:
# Are the means the same?
mean.a <- ped.clean.add.qt$Pheno[ ped.clean.add.cc$FID == 'AFR' ]
mean.e <- ped.clean.add.qt$Pheno[ ped.clean.add.cc$FID == 'EUR' ]
t.test( mean.a, mean.e )

#### What can we do?

In [ ]:
# Let's analyze the ancestry using principal components analysis
# the next couple of steps take a while to run
pca <- prcomp( ped.clean.add[ ,7:998 ], rank=10 )
pcs <- cbind( ped.clean.add[ ,1:2], pca$x[ ,1:10 ] )

head( pcs )

In [ ]:
par( mfrow=c(2,3) )
plot( pcs$PC1, pcs$PC2 )
plot( pcs$PC3, pcs$PC4 )
plot( pcs$PC5, pcs$PC6 )
plot( pcs$PC7, pcs$PC8 )
plot( pcs$PC9, pcs$PC10 )

In [ ]:
## Looks like some clear structure in the genotypes.  Is it ancestry?

In [ ]:
# Color by African (red) vs. European (blue)

par( mfrow=c(2,3) )
plot( pcs$PC1, pcs$PC2, col=c( "red", "blue" )[ 1 + 1*(pcs$FID == 'EUR') ] )
plot( pcs$PC3, pcs$PC4, col=c( "red", "blue" )[ 1 + 1*(pcs$FID == 'EUR') ] )
plot( pcs$PC5, pcs$PC6, col=c( "red", "blue" )[ 1 + 1*(pcs$FID == 'EUR') ] )
plot( pcs$PC7, pcs$PC8, col=c( "red", "blue" )[ 1 + 1*(pcs$FID == 'EUR') ] )
plot( pcs$PC9, pcs$PC10, col=c( "red", "blue" )[ 1 + 1*(pcs$FID == 'EUR') ] )

In [ ]:
## Looks like some clear structure in the genotypes.  Is it ancestry?
## Looks like it.

In [ ]:
## Let's restrict to one population and check for structure

ped.clean.add.eur <- ped.clean.add[ ped.clean.add$FID == 'EUR' ]
pca.eur <- prcomp( ped.clean.add.eur[ ,7:998 ], rank=10 )
pcs.eur <- cbind( ped.clean.add.eur[ ,1:2], pca.eur$x[ ,1:10 ] )

In [ ]:
par( mfrow=c(2,3) )
plot( pcs.eur$PC1, pcs.eur$PC2 )
plot( pcs.eur$PC3, pcs.eur$PC4 )
plot( pcs.eur$PC5, pcs.eur$PC6 )
plot( pcs.eur$PC7, pcs.eur$PC8 )
plot( pcs.eur$PC9, pcs.eur$PC10 )

In [ ]:
## Are we still worrid about structure?
## We can add covariats to our model to help even more.

#### Load covariates

In [ ]:
covs <- fread( './data/QuantitativeTrait_cov_eur.txt' )

In [ ]:
dim( covs )
head( covs )		# PCs are the same conceptually as the last step, just optimized to
					# these particualr simulations

In [ ]:
dim( ped.clean.add.qt)

In [ ]:
## We will merge our ped data with the covs, and do new analysis.  This will
## select European subjects only (not the different number of rows)
ped.clean.add.qt.covs <- merge( covs, ped.clean.add.qt, by=c( "FID", "IID", "Sex" ) )

In [ ]:
dim( ped.clean.add.qt.covs)
head( ped.clean.add.qt.covs[ ,1:25] )

## Perform GWAS 2.0: linear regression

In [ ]:
# Test SNPs: rs707582, rs9913145, rs7237102, rs7741565

In [ ]:
## Quantitative trait
## Linear regression
# rs707582, rs9913145, rs7237102, rs7741565

In [ ]:
snp.lm.covs <- lm( Pheno ~ rs707582 + Sex + PC1 + PC2 + PC3 + PC4 + PC5 + 
									PC6 + PC7 + PC8 + PC9 + PC10, data=ped.clean.add.qt.covs )
summary( snp.lm.covs )

In [ ]:
######ERROR#####
# Make a descriptive plot
plot( ped.clean.add.qt.covs$rs707582, ped.clean.add.qt.covs$Pheno )
# Mean of 0's
points( 0, mean( ped.clean.add.qt.covs$Pheno[ ped.clean.add.qt.covs$ == 0 ] ), 
				col=2, pch=15 )
# Mean of 1's
points( 1, mean( ped.clean.add.qt.covs$Pheno[ ped.clean.add.qt.covs$ == 1 ] ), 
				col=2, pch=15 )
# Mean of 2's
points( 2, mean( ped.clean.add.qt.covs$Pheno[ ped.clean.add.qt.covs$  == 2 ] ), 
				col=2, pch=15 )
#Line from mean of 0's to mean of 2's -> Why? What are we looking for?
segments( 0, mean( ped.clean.add.qt.covs$Pheno[ ped.clean.add.qt.covs$  == 0 ] ),
				2, mean( ped.clean.add.qt.covs$Pheno[ ped.clean.add.qt.covs$  == 2 ] ), 
				col=2 )
# Add regression line
abline( snp.lm.covs$coefficients[1], snp.lm.covs$coefficients[2], col='blue' )


In [ ]:
# Try a few random SNPs 
# How good is our additive model fit our data? 
# What about the "pure additive model"?
# How do these results for our test SNPs compare to previous SNP tests?
# Are the p-values similaar or different?

In [ ]:
# rs707582
# rs9913145
# rs7237102
# rs7741565

## Here is the full genome results in EUR with COVs:

In [ ]:
sumstats.eur <- fread( './data/EUR_COVS_gwas.assoc.linear' )

In [ ]:
dim( sumstats.eur )
head( sumstats.eur )

In [ ]:
sum( sumstats.eur$P < 0.05 )
sum( sumstats.eur$P < 5e-8 )

#### How do these numbers compare to previous ones?

In [ ]:
# Qick Manhattan
sumstats.eur$ManPos <- cumsum( sumstats.eur$BP/1000 )
chr.mids <- cbind( 1:22, NA )
for ( i in 1:22 ) {
	chr.start <- min( sumstats.eur$ManPos[ sumstats.eur$CHR == i ] )	
	chr.stop <- max( sumstats.eur$ManPos[ sumstats.eur$CHR == i ] )
	chr.length <- chr.stop-chr.start
	chr.mids[ i,2 ] <- chr.start + chr.length/2
}

In [ ]:
plot( sumstats.eur$ManPos[ sumstats.eur$P < 0.1 ], 
		-log10( sumstats.eur$P[ sumstats.eur$P < 0.1 ] ),
		xaxt='n', pch=16,
		col=c( 'black','grey' )[ sumstats.eur$CHR[ sumstats.eur$P < 0.1 ] %% 2 + 1 ],
		xlab="Genome Position", ylab="-log10(p)",
		ylim=c(0,max( -log10( sumstats.eur$P ) )+1 ) )
abline( h=-log10( 0.05 ), col='green' )
abline( h=-log10( 5e-8 ), col='green' )
axis( 1, at=chr.mids[,2], labels=chr.mids[,1] )

In [ ]:
# Qick qq-plot
observed.p <- -log10( sumstats.eur$P[ order( sumstats.eur$P ) ] )
expected.p <- -log10( ( 1:length( sumstats.eur$P ) ) / ( length( sumstats.eur$P ) ) )
plot( expected.p[ expected.p < 4 ], observed.p[ expected.p < 4 ], 
		type='l', lwd=6,
		xlim=c( 0, max(expected.p)+1 ), ylim=c( 0, max(observed.p)+1 ) )
points( expected.p[ expected.p >= 4 ], observed.p[ expected.p >= 4 ],
		pch=16 )
abline( 0,1 )

In [ ]:
## What do we think of these plots?
## Does this look good or problematic?